In [140]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
import socket

In [103]:
# First method : getting into frame and find out the location of addr
driver = webdriver.Chrome(r'D:\Download\ChromeDriver\chromedriver_win32 (1)\chromedriver')
driver.get('https://centralops.net/co/')

url = 'www.cscglobal.com'

try:
    driver.switch_to.frame('main')
    elem_addr = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/dl/dd[1]/form/div/label/input')
    time.sleep(0.5)
    elem_addr.clear()
    elem_addr.send_keys(url)
    time.sleep(0.5)
    elem_addr.send_keys(Keys.RETURN)
except:
    driver.close()
    
page_source = driver.page_source

print(driver.page_source)

<html xmlns="http://www.w3.org/1999/xhtml"><head>
    
<title>www.cscglobal.com - Domain Dossier - owner and registrar information, whois and DNS records</title>
<meta http-equiv="Content-Type" content="text/html;charset=UTF-8" />
<meta name="robots" content="nofollow,noindex,noarchive" />		

<link href="styles/Utility.css" type="text/css" rel="stylesheet" />

  </head>
	<body>
		<table class="form" cellspacing="0" cellpadding="7" border="0">
			<tbody><tr>
				<td class="title" valign="baseline" width="0"><span class="name">Domain Dossier</span></td>
				<td class="title" valign="baseline" align="right" width="0"><span class="desc">Investigate
						domains and IP addresses</span></td></tr>
			<tr>
				<td class="form" valign="top" colspan="2" height="0%"><form name="dd" method="post" action="/co/DomainDossier.aspx">
<input type="hidden" name="__VIEWSTATE" id="__VIEWSTATE" value="" />

      <table cellspacing="0" cellpadding="5" width="100%" border="0">
        <tbody><tr>
          

In [157]:
extract_data(page_source)

{'Registrant Name': ' Domain Administrator',
 'Registrant Organization': ' CSC Corporate Domains, Inc.',
 'Registrant Street': ' 251 Little Falls Drive',
 'Registrant City': ' Wilmington',
 'Registrant State/Province': ' DE',
 'Registrant Postal Code': ' 19808',
 'Registrant Country': ' US',
 'Registrant Phone': ' +1.3026365400',
 'Registrant Phone Ext': ' ',
 'Registrant Fax': ' +1.3026365454',
 'Registrant Fax Ext': ' ',
 'Registrant Email': ' admin@internationaladmin.com',
 'registrar': 'CSC Corporate Domains, Inc.',
 'registrar_abuse_email': 'domainabuse@cscglobal.com',
 'IP Address': '165.160.32.200',
 'Host': 'CSC Corporate Domains, Inc.'}

In [156]:
def extract_data(page_source):
    
    if page_source is None:
        return
    
    # Registrar data
    registrar = re.search(re.compile(r'Registrar:(.*)'), page_source).group(1)
    if registrar:
        registrar = str.strip(registrar)
    else:
        return

    # Registrar abuse email
    registrar_email = re.search(re.compile(r'Registrar Abuse Contact Email:(.*)'), page_source).group(1)
    if registrar_email:
        registrar_email = str.strip(registrar_email)
    else:
        return
    
    # Domain Name
    domain_name = re.search(re.compile(r'Domain Name:(.*)'), page_source).group(1)
    if domain_name:
        domain_name = str.lower(str.strip(domain_name))
    else:
        return
        
    # Registrant Info
    registrant = re.findall(re.compile(r'(Registrant.*):(.*)'), page_source)
    if registrant is not None:
        dic = dict((x,y) for x,y in registrant)
        del dic['Registrant ID']
    else:
        return
    
    # IP
    ip_addr = socket.gethostbyname(domain_name)
    
    # host name
    organization = re.search(re.compile(r'Organization:(.*)'), page_source).group(1)
    if organization is not None:
        host_name = str.strip(organization)
    else:
        return
    
    dic_alt = {}
    if dic is not None:
        dic['registrar'] = registrar
        dic['registrar_abuse_email'] = registrar_email
        dic['IP Address'] = ip_addr
        dic['Host'] = host_name
        return dic
    else:
        dic_alt['registrar'] = registrar
        dic_alt['registrar_abuse_email'] = registrar_email
        dic_alt['IP Address'] = ip_addr
        dic_alt['Host'] = host_name
        return dic_alt